In [102]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [103]:
import netCDF4 as nc
import gsw
import pickle as pk
import numpy as np

In [104]:
# load temperature and salinity from SODA file
SODA_file = "/d1/enrique/bdry/NWA_bdry_SODA3.3.1_y2012.nc"
Sds = nc.Dataset(SODA_file)
Sds.variables.keys()

odict_keys(['theta_s', 'theta_b', 'Tcline', 'hc', 's_rho', 's_w', 'Cs_r', 'Cs_w', 'h', 'ocean_time', 'zeta_south', 'zeta_east', 'temp_east', 'temp_south', 'salt_east', 'salt_south', 'angle', 'lat_psi', 'lat_rho', 'lat_u', 'lat_v', 'lon_psi', 'lon_rho', 'lon_u', 'lon_v', 'mask_psi', 'mask_rho', 'mask_u', 'mask_v', 'pm', 'pn', 'u_east', 'u_south', 'ubar_east', 'ubar_south', 'v_east', 'v_south', 'vbar_east', 'vbar_south'])

In [105]:
temp_south = Sds.variables['temp_south'][:]
temp_east = Sds.variables['temp_east'][:]

salt_south = Sds.variables['salt_south'][:]
salt_east = Sds.variables['salt_east'][:]

#temp_south.shape
#temp_east.shape
#salt_south.shape
salt_east.shape

# east is 362, south is 722

(73, 40, 362)

In [106]:
# I calculated the depth from the grid file and pickled
# load that

with open('/d1/mkelly/NWA/romsdepth.p', 'rb') as f:
    depth = pk.load(f)

depth_east = depth[:, 721, :]
depth_south = depth[1, :, :]

# horz x vert
# may need to switch dims to match
#depth_east.shape
depth_south.shape

(722, 40)

In [107]:
# get latitude and longitude coordinates from NWA grid file
grd_file = "/d1/mkelly/NWA/NWA_grd.nc"
grdds = nc.Dataset(grd_file)

lat = grdds.variables['lat_rho'][:]
lon = grdds.variables['lon_rho'][:]

lon.shape

(362, 722)

In [108]:
# index BC coords

lat_south = lat[0, :]
lon_south = lon[0, :]

lat_east = lat[:, 721]
lon_east = lon[:, 721]

lon_south.shape
#lat_east.shape

(722,)

In [115]:
# pull out one time step

salt_south1 = salt_south[0, :, :]
salt_east1 = salt_east[0, :, :]

temp_south1 = temp_south[0, :, :]
temp_east1 = temp_east[0, :, :]

#salt_south1.shape
temp_south1.shape

# can put this in a loop later?

(40, 722)

In [110]:
# need absolute salinity

#SA_south = gsw.SA_from_SP(SP, p, lon, lat)

# inputs are SP, p, lon, lat
# should all have the same dims
# every point has salinity, pressure, lon, and lat

# dims

# SP: 40, 722
# p: 722, 40
# lon: 722
# lat: 722

In [111]:
# repeat lat and lon at all 40 depth levels

lon_south = np.tile(lon_south, (40, 1))
lat_south = np.tile(lat_south, (40, 1))

#lon_south.shape
lat_south.shape

(40, 722)

In [112]:
# SP: 40, 722
# p: 722, 40
# lon: 40, 722
# lat: 40, 722

In [113]:
# now need to flip p

depth_south=np.transpose(depth_south)
depth_east=np.transpose(depth_east)

depth_south.shape

(40, 722)

In [114]:
# calculate absolute salinity

SA_south = gsw.SA_from_SP(salt_south1, depth_south, lon_south, lat_south)
SA_east = gsw.SA_from_SP(salt_east1, depth_east, lon_east, lat_east)

SA_south.shape

(40, 722)

In [117]:
# potential to conservative temperature

CT_south = gsw.CT_from_pt(SA_south, temp_south1)
CT_east = gsw.CT_from_pt(SA_east, temp_east1)

In [ ]:
# conservative to absolute temperature

